##　セグメンテーション
9から12章では分類機のモデル訓練に使用するデータは人手でアノテーションしている。
モデルへの入力の結節を自動で生成したい。

現実世界のアプローチでは複数の問題を個々のステップで解決するが、ディープラーニングの研究では複数の問題から構成されている問題を単一のモデルによって解決させる傾向がある。

１３章ではCTスキャンの元データから結節である可能性がある領域を全て見つける。
結節の一部であるボクセルに対してラベルをつけるセグメンテーション処理を行う。

セマンティックセグメンテーション

画像内の個々のピクセルに対してラベルをつける。今回は結節にTrue、正常組織にFalseのラベルをつける。


物体検出

画像内の対象にバウンディングボックスを設定する
こちらの方が計算リソースが必要である

## U-NET

分類タスクでは画像を畳み込みとダウンサンプリングを繰り返し、各クラスの確率のベクトルにする。

セグメンテーションでは入力と出力のサイズは同じにしたい。畳み込みによりテクスチャや色を検出し、ダウンサプリングによって畳み込みの受容野を広げ局所だけでなく広い特徴を掴む。このようにしていくと画像サイズが小さくなっていくので、1つのピクセルをn＊nのブロックに置き換えるアップサンプリングを行う。

さらに今回はパディングをおこなことで、画像周辺のピクセルが失われないようにして、かつUのダウンサンプリング時とアップサンプリング時のサイズが同じになる。

スキップ接続がないと、ダウンサンプリング時に画像が小さくなり物体境界の正確な位置情報が失われやすくなる。

U-Netのスキップ接続はResNetのスキップ接続とは異なり、ダウンサンプリング側の入力を対応する出力側のアップサンプリング側へつなぐ。
このことにより、Uの底で広い受容野の情報とネットワーク初期の入力に近い高精細な情報の両方を出力へ繋げていく。

既存コードの再利用は良いアイデアだが、どのようなモデルで、どのような実装、訓練か取り組んでいるプロジェクトに適用できる部分があるか把握しておくことが必要。

既存のモデルを変更していく場合は、1つずつ変更し比較していくと良い。

今回は既存のU-NETから
1：入力をバッチ正規化する
2：出力の前にnn.sigmoidを使い[0,1]の範囲にする
3：モデルの深さとフィルタを減らす

In [6]:
class UNetWrapper(nn.Module):
    def __init__(self, **kwargs):
        # kwargs はコンストラクタに渡される全てのキーワード引数を含む辞書
        super().__init__()

        # BatchNorm2d は入力のチャンネル数を必要とする
        # その情報をキーワード引数から取り出す
        self.input_batchnorm = nn.BatchNorm2d(kwargs["in_channels"])
        # U-Netの取り込み部分はこれだけだが、ほとんどの処理はここで行われる
        self.unet = UNet(**kwargs)
        self.final = nn.Sigmoid()

        # 第11章と同じように独自の重み初期化を行う
        self._init_weights()
        
    def _init_weights(self):
        init_set = {
            nn.Conv2d,
            nn.Conv3d,
            nn.ConvTranspose2d,
            nn.ConvTranspose3d,
            nn.Linear,
        }
        for m in self.modules():
            if type(m) in init_set:
                nn.init.kaiming_normal_(
                    m.weight.data, mode="fan_out", nonlinearity="relu", a=0
                )
                if m.bias is not None:
                    fan_in, fan_out = nn.init._calculate_fan_in_and_fan_out(
                        m.weight.data
                    )
                    bound = 1 / math.sqrt(fan_out)
                    nn.init.normal_(m.bias, -bound, bound)

        # nn.init.constant_(self.unet.last.bias, -4)
        # nn.init.constant_(self.unet.last.bias, 4)

    def forward(self, input_batch):
        bn_output = self.input_batchnorm(input_batch)
        un_output = self.unet(bn_output)
        fn_output = self.final(un_output)
        return fn_output



NameError: name 'nn' is not defined

今回の画像は三次元だが、nn.Batchnormは2d
メモリ使用量を減らすためだが、前後の画像の情報は検出には必要。
二次元画像として処理してセグメンテーション処理時には三次元画像として渡す。

モデル学習時に自ら隣接していることを学ぶ必要があるが、z軸の画像量が少なく容易と考える。

画像のチャネルをスライス（＋2、＋1、0、-1、-2）に絞り、スライス、x軸、y軸の入力とする。
スライス方向の情報を限定的にするが、結節の大きさは小さく今回の問題では十分だと判断。

## モデルの設計

どのようなトレードオフを考えなけらばいけないかはフローチャートや経験則はないが、体系的に仮説を検証することが大切であり、
思いつきの変更等は堪え、複数の変更を同時にテストすることはだめ。

## 正解データの作成

バウンディングボックスを作成し、その後マスクとする

結節の中心の位置はわかっているから、閾値以下になるまで左右、上下に探索し閾値以下になったらそこまでの範囲とする。
他の組織と隣接している可能性もあるので、片方が低密度に触れたら探索終了となる。片方ずつの独立の探索はできず、結節は中心の情報が必要。

下記のループ処理後にバウンディングボックス内の閾値より高い領域を調理和として取り出し、マスクとする。

In [ ]:
def buildAnnotationMask(self, positiveInfo_list, threshold_hu=-700):
        # hu_aと同じ次元のゼロarrayを作成
        boundingBox_a = np.zeros_like(self.hu_a, dtype=np.bool)

        for candidateInfo_tup in positiveInfo_list:
            center_irc = xyz2irc(
                candidateInfo_tup.center_xyz,
                self.origin_xyz,
                self.vxSize_xyz,
                self.direction_a,
            )
            
            # 結節中心の位置情報
            ci = int(center_irc.index)
            cr = int(center_irc.row)
            cc = int(center_irc.col)

            index_radius = 2
            try:
                while (
                    self.hu_a[ci + index_radius, cr, cc] > threshold_hu
                    and self.hu_a[ci - index_radius, cr, cc] > threshold_hu
                ):
                    index_radius += 1
            except IndexError:
                index_radius -= 1
                
            row_radius = 2
            try:
                while (
                    self.hu_a[ci, cr + row_radius, cc] > threshold_hu
                    and self.hu_a[ci, cr - row_radius, cc] > threshold_hu
                ):
                    row_radius += 1
            except IndexError:
                row_radius -= 1

            col_radius = 2
            try:
                while (
                    self.hu_a[ci, cr, cc + col_radius] > threshold_hu
                    and self.hu_a[ci, cr, cc - col_radius] > threshold_hu
                ):
                    col_radius += 1
            except IndexError:
                col_radius -= 1
                
                
            
            boundingBox_a[
                ci - index_radius : ci + index_radius + 1,
                cr - row_radius : cr + row_radius + 1,
                cc - col_radius : cc + col_radius + 1,
            ] = True
        
        # バウンディングボックスからマスクをくり抜く
        mask_a = boundingBox_a & (self.hu_a > threshold_hu)

        return mask_a

In [51]:
import numpy as np
x = np.ones(27)
x = x.reshape((3, 3, 3))
x

a=np.zeros_like(x,dtype=np.bool)
a

/tmp/ipykernel_221/1484239419.py:6: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  a=np.zeros_like(x,dtype=np.bool)


array([[[False, False, False],
        [False, False, False],
        [False, False, False]],

       [[False, False, False],
        [False, False, False],
        [False, False, False]],

       [[False, False, False],
        [False, False, False],
        [False, False, False]]])

In [52]:
a[1:2, 0:2, 1:2]=True

In [53]:
a

array([[[False, False, False],
        [False, False, False],
        [False, False, False]],

       [[False,  True, False],
        [False,  True, False],
        [False, False, False]],

       [[False, False, False],
        [False, False, False],
        [False, False, False]]])

## データセット

生成するデータは複数のチャネルを持つ二次元画像。結節として注目しているスライス断面に隣接しているスライス画像をチャネルに割り当てる。一つのデータのスライスは数枚になる。

訓練データと検証データのサイズが異なる

今回はフルサイズのデータで訓練したら成績はよくなかった。これは画像全体に比較して結節のサイズが小さく、陽性サンプルが陰性サンプルに埋もれてしまう現象と同じことが起きたと考えられる。そのため訓練時は結節の周囲のみクロップした画像で学習することで陰性サンプルに対して陽性サンプルの割合を増やす。

セグメンテーションモデルではピクセル単位で処理をするため、任意の画像サイズを扱うことができ、訓練と検証で画像サイズが異なってもよい。検証では訓練と同じ重みの畳み込み計算を、より多くのピクセルを持つ大きな画像に対し適用する。

In [ ]:
def __init__(
        self,
        val_stride=0,
        isValSet_bool=None,
        series_uid=None,
        contextSlices_count=3,
        fullCt_bool=False,
    ):
        self.contextSlices_count = contextSlices_count
        self.fullCt_bool = fullCt_bool

検証では結節のあるなし関係なく全ての画像リストからフルサイズで画像を取得、
contextSlices_countは適当に指定し、指定した結節の断面の上下数枚のデータを取得する。


In [ ]:
# データ取得（検証）
#　
def __getitem__(self, ndx):
        series_uid, slice_ndx = self.sample_list[ndx % len(self.sample_list)]
        return self.getitem_fullSlice(series_uid, slice_ndx)
    
def getitem_fullSlice(self, series_uid, slice_ndx):
        ct = getCt(series_uid)
        ct_t = torch.zeros((self.contextSlices_count * 2 + 1, 512, 512))

        start_ndx = slice_ndx - self.contextSlices_count
        end_ndx = slice_ndx + self.contextSlices_count + 1
        for i, context_ndx in enumerate(range(start_ndx, end_ndx)):
            context_ndx = max(context_ndx, 0)
            context_ndx = min(context_ndx, ct.hu_a.shape[0] - 1)
            ct_t[i] = torch.from_numpy(ct.hu_a[context_ndx].astype(np.float32))

        # CTs are natively expressed in https://en.wikipedia.org/wiki/Hounsfield_scale
        # HU are scaled oddly, with 0 g/cc (air, approximately) being -1000 and 1 g/cc (water) being 0.
        # The lower bound gets rid of negative density stuff used to indicate out-of-FOV
        # The upper bound nukes any weird hotspots and clamps bone down
        ct_t.clamp_(-1000, 1000)

        pos_t = torch.from_numpy(ct.positive_mask[slice_ndx]).unsqueeze(0)

        return ct_t, pos_t, ct.series_uid, slice_ndx


訓練では結節のリストから7枚で画像サイズは中心から96*96で取得する。
その後ランダムに64*64にクロップする

In [ ]:
# データ取得（訓練）

def __getitem__(self, ndx):
        candidateInfo_tup = self.pos_list[ndx % len(self.pos_list)]
        return self.getitem_trainingCrop(candidateInfo_tup)

def getitem_trainingCrop(self, candidateInfo_tup):
        ct_a, pos_a, center_irc = getCtRawCandidate(
            candidateInfo_tup.series_uid,
            candidateInfo_tup.center_xyz,
            (7, 96, 96),
        )
        
        # スライスは固定
        pos_a = pos_a[3:4]

        row_offset = random.randrange(0, 32)
        col_offset = random.randrange(0, 32)
        ct_t = torch.from_numpy(
            ct_a[:, row_offset : row_offset + 64, col_offset : col_offset + 64]
        ).to(torch.float32)
        pos_t = torch.from_numpy(
            pos_a[:, row_offset : row_offset + 64, col_offset : col_offset + 64]
        ).to(torch.long)

        slice_ndx = center_irc.index

        return ct_t, pos_t, candidateInfo_tup.series_uid, slice_ndx



## データオーギュメンテーションはGPUで

ボトルネックは一般的に下記で

1：データ読み込みパイプライン中のデータ展開時（RAM）

2：CPUでのデータ前処理（正規化、オーギュメンテーション）

3：GPUでの訓練ループ
ここがボトルネックになるようにする

内容は12章と同じ


##　訓練

モデルのインスタンス化（セグメンテーションモデルとオーギュメンテーションモデル）

In [ ]:
def initModel(self):
        segmentation_model = UNetWrapper(
            in_channels=7,
            n_classes=1,
            depth=3,
            wf=4,
            padding=True,
            batch_norm=True,
            up_mode='upconv',
        )

        augmentation_model = SegmentationAugmentation(**self.augmentation_dict)

        if self.use_cuda:
            log.info("Using CUDA; {} devices.".format(
                torch.cuda.device_count()))
            if torch.cuda.device_count() > 1:
                segmentation_model = nn.DataParallel(segmentation_model)
                augmentation_model = nn.DataParallel(augmentation_model)
            segmentation_model = segmentation_model.to(self.device)
            augmentation_model = augmentation_model.to(self.device)

        return segmentation_model, augmentation_model

## 最適化

Adamはパラメータごとに学習率を調整し更新していく。基本的に学習率はデフォルト値以外を使用。

SGDの方が成績が良い場合もあるがハイパーパラメータの探索に時間がかかる。

In [ ]:
def initOptimizer(self):
        return Adam(self.segmentation_model.parameters())
        # return SGD(self.segmentation_model.parameters(), lr=0.001, momentum=0.99)

## 損失関数

Dice係数

画像全体に対して比較的狭い領域だけが陽性になる場合に使用。ピクセル単位のF1値のようなもの。
Dice係数は高い方が良い値で（最大1）、pytorchでは1ーDice係数として損失に利用。

In [ ]:
def diceLoss(self, prediction_g, label_g, epsilon=1):
        diceLabel_g = label_g.sum(dim=[1, 2, 3])
        dicePrediction_g = prediction_g.sum(dim=[1, 2, 3])
        diceCorrect_g = (prediction_g * label_g).sum(dim=[1, 2, 3])

        diceRatio_g = (2 * diceCorrect_g + epsilon) \
            / (dicePrediction_g + diceLabel_g + epsilon)

        return 1 - diceRatio_g


Dice係数では

正解U予測＊２　/  正解＋予測

医療では偽陰性は危険。なので再現率（recall)を重要視したい

Diceの入力を変更し、　正解U予測　/ 予測　（真陽性　/ 予測）

In [ ]:
def computeBatchLoss(self, batch_ndx, batch_tup, batch_size, metrics_g,
                         classificationThreshold=0.5):
        input_t, label_t, series_list, _slice_ndx_list = batch_tup

        input_g = input_t.to(self.device, non_blocking=True)
        label_g = label_t.to(self.device, non_blocking=True)

        if self.segmentation_model.training and self.augmentation_dict:
            input_g, label_g = self.augmentation_model(input_g, label_g)

        prediction_g = self.segmentation_model(input_g)

        diceLoss_g = self.diceLoss(prediction_g, label_g)
        fnLoss_g = self.diceLoss(prediction_g * label_g, label_g)

        start_ndx = batch_ndx * batch_size
        end_ndx = start_ndx + input_t.size(0)

        with torch.no_grad():
            predictionBool_g = (prediction_g[:, 0:1]
                                > classificationThreshold).to(torch.float32)

            tp = (predictionBool_g * label_g).sum(dim=[1, 2, 3])
            fn = ((1 - predictionBool_g) * label_g).sum(dim=[1, 2, 3])
            fp = (predictionBool_g * (~label_g)).sum(dim=[1, 2, 3])

            metrics_g[METRICS_LOSS_NDX, start_ndx:end_ndx] = diceLoss_g
            metrics_g[METRICS_TP_NDX, start_ndx:end_ndx] = tp
            metrics_g[METRICS_FN_NDX, start_ndx:end_ndx] = fn
            metrics_g[METRICS_FP_NDX, start_ndx:end_ndx] = fp

        return diceLoss_g.mean() + fnLoss_g.mean() * 8

Dice係数にて二つの指標を求めて、recallは８倍して重みをつけて返す。

重みが強いので、SGDで最適化すると全てのピクセルを陽性と判断してしまう。
Adamにすると学習率を細かく調整してくれるので、偽陰性による損失に過剰に引っ張られなくすむ。

## 画像の出力(Tensorboad)

セグメンテーションタスクでは出力を視覚的に確認できる。TensorBoardでは画像にも対応している。
関数の表示は　logImage 関数を用いる。

訓練においてバランスの調整に注意する。

１：短時間で、訓練がうまくいっているのか大まかに把握する。

評価指標を表示するlogMetricsを頻繁に実行する。

２：GPUの計算時間の多くを検証ではなく、訓練に使う

logMetricsで検証する回数を減らす

３：検証セットで訓練が正しく進歩しているかを把握する。

定期的な検証を行う

In [ ]:
# 初回と5エポックごとに行う
# CTは六枚を選んで表示する。

def main(self):
        log.info("Starting {}, {}".format(type(self).__name__, self.cli_args))

        train_dl = self.initTrainDl()
        val_dl = self.initValDl()

        best_score = 0.0
        self.validation_cadence = 5
        for epoch_ndx in range(1, self.cli_args.epochs + 1):
            log.info("Epoch {} of {}, {}/{} batches of size {}*{}".format(
                epoch_ndx,
                self.cli_args.epochs,
                len(train_dl),
                len(val_dl),
                self.cli_args.batch_size,
                (torch.cuda.device_count() if self.use_cuda else 1),
            ))

            trnMetrics_t = self.doTraining(epoch_ndx, train_dl)
            self.logMetrics(epoch_ndx, 'trn', trnMetrics_t)

            if epoch_ndx == 1 or epoch_ndx % self.validation_cadence == 0:
                # if validation is wanted
                valMetrics_t = self.doValidation(epoch_ndx, val_dl)
                score = self.logMetrics(epoch_ndx, 'val', valMetrics_t)
                best_score = max(score, best_score)

                self.saveModel('seg', epoch_ndx, score == best_score)

                self.logImages(epoch_ndx, 'trn', train_dl)
                self.logImages(epoch_ndx, 'val', val_dl)


In [145]:
import numpy as np

a=np.ones([3,3,3])
a[0][1]=2
p=np.ones([1,3,3,3])
p[0][1][1]=2

In [146]:
import torch
a= torch.from_numpy(a)
p= torch.from_numpy(p)


In [147]:
a=a.unsqueeze(0)

In [148]:
a = a[0] >1
a

tensor([[[False, False, False],
         [ True,  True,  True],
         [False, False, False]],

        [[False, False, False],
         [False, False, False],
         [False, False, False]],

        [[False, False, False],
         [False, False, False],
         [False, False, False]]])

In [149]:
p=p.unsqueeze(0)
p =p[0][0] >1
p

tensor([[[False, False, False],
         [False, False, False],
         [False, False, False]],

        [[False, False, False],
         [ True,  True,  True],
         [False, False, False]],

        [[False, False, False],
         [False, False, False],
         [False, False, False]]])

In [150]:
p=p.numpy()
a=a.numpy()

In [151]:
p &  a

array([[[False, False, False],
        [False, False, False],
        [False, False, False]],

       [[False, False, False],
        [False, False, False],
        [False, False, False]],

       [[False, False, False],
        [False, False, False],
        [False, False, False]]])

画像の表示

In [152]:
# モデルを推論に切り替える
# 毎回同じ１２を確認する。そのためrシャッフルされている可能性のあるリストをソートする


def logImages(self, epoch_ndx, mode_str, dl):
        self.segmentation_model.eval()

        images = sorted(dl.dataset.series_list)[:12]
        for series_ndx, series_uid in enumerate(images):
            ct = getCt(series_uid)

            for slice_ndx in range(6):
                ct_ndx = slice_ndx * (ct.hu_a.shape[0] - 1) // 5
                sample_tup = dl.dataset.getitem_fullSlice(series_uid, ct_ndx)
                # 出力はct_t, pos_t, ct.series_uid, slice_ndx

                ct_t, label_t, series_uid, ct_ndx = sample_tup

                input_g = ct_t.to(self.device).unsqueeze(0)
                label_g = pos_g = label_t.to(self.device).unsqueeze(0)
                # Unetのラッパーへ入力はバッチを追加、出力はprediction_g
                prediction_g = self.segmentation_model(input_g)[0]
                # バッチを追加したから一つ目を
                prediction_a = prediction_g.to('cpu').detach().numpy()[0] > 0.5
                # 追加したバッチと、チャネルの一次元目を（今回は一次元しかない）
                label_a = label_g.cpu().numpy()[0][0] > 0.5

                ct_t[:-1, :, :] /= 2000
                ct_t[:-1, :, :] += 0.5

                ctSlice_a = ct_t[dl.dataset.contextSlices_count].numpy()

                # カラーで表示する
                # 白黒を三つにコピーしてRGBとする
                
                image_a = np.zeros((512, 512, 3), dtype=np.float32)
                image_a[:, :, :] = ctSlice_a.reshape((512, 512, 1))
                # 偽陽性は赤
                image_a[:, :, 0] += prediction_a & (1 - label_a)
                # 偽陰性は０と１に値が入り、オレンジで
                image_a[:, :, 0] += (1 - prediction_a) & label_a
                image_a[:, :, 1] += ((1 - prediction_a) & label_a) * 0.5
                # 真陽性は緑
                image_a[:, :, 1] += prediction_a & label_a
                image_a *= 0.5
                
                # データオーギュメンテーションで値が範囲外になっている可能性があり、再度クリップ
                image_a.clip(0, 1, image_a)

                writer = getattr(self, mode_str + '_writer')
                writer.add_image(
                    f'{mode_str}/{series_ndx}_prediction_{slice_ndx}',
                    image_a,
                    self.totalTrainingSamples_count,
                    dataformats='HWC',
                )

                if epoch_ndx == 1:
                    image_a = np.zeros((512, 512, 3), dtype=np.float32)
                    image_a[:, :, :] = ctSlice_a.reshape((512, 512, 1))
                    # image_a[:,:,0] += (1 - label_a) & lung_a # Red
                    image_a[:, :, 1] += label_a  # Green
                    # image_a[:,:,2] += neg_a  # Blue

                    image_a *= 0.5
                    image_a[image_a < 0] = 0
                    image_a[image_a > 1] = 1
                    writer.add_image(
                        '{}/{}_label_{}'.format(
                            mode_str,
                            series_ndx,
                            slice_ndx,
                        ),
                        image_a,
                        self.totalTrainingSamples_count,
                        # 出力そのままは"CHW"pytorchは"HWC"で三つ目にカラーがあることを表記する
                        dataformats='HWC',
                    )
                # This flush prevents TB from getting confused about which
                # data item belongs where.
                writer.flush()


In [ ]:
評価はF1とrecallで確認するが、偽陰性を無くしたいので、モデル更新にはrecallの損失を用いる

デメリットとして偽陽性は増加することが考えられるが、三ステップの初めなので次の分類ステップで除外されることを期待する。

In [ ]:
 def logMetrics(self, epoch_ndx, mode_str, metrics_t):
        log.info("E{} {}".format(
            epoch_ndx,
            type(self).__name__,
        ))

        metrics_a = metrics_t.detach().numpy()
        sum_a = metrics_a.sum(axis=1)
        assert np.isfinite(metrics_a).all()

        allLabel_count = sum_a[METRICS_TP_NDX] + sum_a[METRICS_FN_NDX]

        metrics_dict = {}
        metrics_dict['loss/all'] = metrics_a[METRICS_LOSS_NDX].mean()

        metrics_dict['percent_all/tp'] = \
            sum_a[METRICS_TP_NDX] / (allLabel_count or 1) * 100
        metrics_dict['percent_all/fn'] = \
            sum_a[METRICS_FN_NDX] / (allLabel_count or 1) * 100
        metrics_dict['percent_all/fp'] = \
            sum_a[METRICS_FP_NDX] / (allLabel_count or 1) * 100

        precision = metrics_dict['pr/precision'] = sum_a[METRICS_TP_NDX] \
            / ((sum_a[METRICS_TP_NDX] + sum_a[METRICS_FP_NDX]) or 1)
        recall = metrics_dict['pr/recall'] = sum_a[METRICS_TP_NDX] \
            / ((sum_a[METRICS_TP_NDX] + sum_a[METRICS_FN_NDX]) or 1)

        metrics_dict['pr/f1_score'] = 2 * (precision * recall) \
            / ((precision + recall) or 1)

        log.info(("E{} {:8} "
                  + "{loss/all:.4f} loss, "
                  + "{pr/precision:.4f} precision, "
                  + "{pr/recall:.4f} recall, "
                  + "{pr/f1_score:.4f} f1 score"
                  ).format(
            epoch_ndx,
            mode_str,
            **metrics_dict,
        ))
        log.info(("E{} {:8} "
                  + "{loss/all:.4f} loss, "
                  + "{percent_all/tp:-5.1f}% tp, {percent_all/fn:-5.1f}% fn, {percent_all/fp:-9.1f}% fp"
                  ).format(
            epoch_ndx,
            mode_str + '_all',
            **metrics_dict,
        ))

        self.initTensorboardWriters()
        writer = getattr(self, mode_str + '_writer')

        prefix_str = 'seg_'

        for key, value in metrics_dict.items():
            writer.add_scalar(prefix_str + key, value,
                              self.totalTrainingSamples_count)

        writer.flush()

        score = metrics_dict['pr/recall']

        return score


In [ ]:
 def saveModel(self, type_str, epoch_ndx, isBest=False):
        file_path = os.path.join(
            'data-unversioned',
            'part2',
            'models',
            self.cli_args.tb_prefix,
            '{}_{}_{}.{}.state'.format(
                type_str,
                self.time_str,
                self.cli_args.comment,
                self.totalTrainingSamples_count,
            )
        )

        os.makedirs(os.path.dirname(file_path), mode=0o755, exist_ok=True)

        model = self.segmentation_model
        if isinstance(model, torch.nn.DataParallel):
            model = model.module

        state = {
            'sys_argv': sys.argv,
            'time': str(datetime.datetime.now()),
            'model_state': model.state_dict(),
            'model_name': type(model).__name__,
            'optimizer_state': self.optimizer.state_dict(),
            'optimizer_name': type(self.optimizer).__name__,
            'epoch': epoch_ndx,
            'totalTrainingSamples_count': self.totalTrainingSamples_count,
        }
        torch.save(state, file_path)

        log.info("Saved model params to {}".format(file_path))

        if isBest:
            best_path = os.path.join(
                'data-unversioned', 'part2', 'models',
                self.cli_args.tb_prefix,
                f'{type_str}_{self.time_str}_{self.cli_args.comment}.best.state')
            shutil.copyfile(file_path, best_path)

            log.info("Saved model params to {}".format(best_path))

        with open(file_path, 'rb') as f:
            log.info("SHA1: " + hashlib.sha1(f.read()).hexdigest())



## 結果の評価

訓練では真陽性は増加、F1も増加、偽陽性と偽陰性は低下していき期待通りの動き。
検証では画像サイズが64から512（64倍）となり、真陽性や偽陰性、偽陽性の比率は大きく変わるはず（陰性のピクセルの割合が大きくなるため）
しかしモデル更新のためのrecallが10エポックから伸びないため過学習が予測される。

ここではNが増えても良いので再現率のみで評価し次に進む。